In [2]:
import mne

dataset1 = mne.read_epochs("../beta_epo1.fif")
dataset2 = mne.read_epochs("../beta_epo2.fif")
# entendimento dos dados
dataset1.get_data().shape, dataset2.get_data().shape

Reading c:\Users\caio-\Documents\facul\patter-recog\SSVEP\beta\classifier\..\beta_epo1.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    2996.00 ms
        0 CTF compensation matrices available
Not setting metadata
160 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\caio-\Documents\facul\patter-recog\SSVEP\beta\classifier\..\beta_epo2.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    2996.00 ms
        0 CTF compensation matrices available
Not setting metadata
160 matching events found
No baseline correction applied
0 projection items activated


C:\Users\caio-\AppData\Local\Temp\ipykernel_22872\2788898231.py:3: RuntimeWarning: This filename (../beta_epo1.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  dataset1 = mne.read_epochs("../beta_epo1.fif")
C:\Users\caio-\AppData\Local\Temp\ipykernel_22872\2788898231.py:4: RuntimeWarning: This filename (../beta_epo2.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  dataset2 = mne.read_epochs("../beta_epo2.fif")


((160, 9, 750), (160, 9, 750))

## Classificação por isolamento de frequência


In [4]:
import numpy as np

hits_dataset1 = 0
hits_dataset2 = 0

y_dataset1 = np.load("../../datasets/beta/labels1.npy") #carrega os labels
y_dataset2 = np.load("../../datasets/beta/labels2.npy")

threshold = 0.20 #threshold de 20%
targets = [float(item) for item in dataset1.event_id.keys()] #pega os targets

for i in range(len(dataset1)): 
    psd_dataset1 = dataset1[i].compute_psd(method='welch', fmin=7, fmax=17, verbose=False) #pega a psd
    psd_dataset2 = dataset2[i].compute_psd(method='welch', fmin=7, fmax=17, verbose=False) #pega a psd

    for target in targets: #para cada target
        fmin = target - threshold #calcula o fmin e fmax
        fmax = target + threshold 

        # Dataset 1
        features_dataset1 = psd_dataset1.get_data(fmin=fmin, fmax=fmax) #pega os dados
        X_dataset1 = np.array(features_dataset1) #transforma em array
        max_frequency_dataset1 = np.max(X_dataset1, axis=1) #pega o maximo de cada linha

        if np.any((max_frequency_dataset1 >= fmin) & (max_frequency_dataset1 <= fmax)): #se o maximo estiver entre o fmin e fmax
            hits_dataset1 += 1 #acertou

        # Dataset 2
        features_dataset2 = psd_dataset2.get_data(fmin=fmin, fmax=fmax) #pega os dados
        X_dataset2 = np.array(features_dataset2) #transforma em array
        max_frequency_dataset2 = np.max(X_dataset2, axis=1) #pega o maximo de cada linha

        if np.any((max_frequency_dataset2 >= fmin) & (max_frequency_dataset2 <= fmax)): #se o maximo estiver entre o fmin e fmax
            hits_dataset2 += 1 #acertou

accuracy_dataset1 = (hits_dataset1 / len(y_dataset1)) * 100  #calcula a acuracia
accuracy_dataset2 = (hits_dataset2  / len(y_dataset2)) * 100

print(f"Accuracia dataset 1: {accuracy_dataset1} %")
print(f"Accuracia dataset 2: {accuracy_dataset2} %")

Accuracia dataset 1: 20.0 %
Accuracia dataset 2: 16.25 %


## Classificação utilizando RFE para seleção de atributos e SVM para classificação

In [12]:
import numpy as np
import mne
from sklearn.svm import SVC
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


ch_ideal = ["Pz", "PO3", "PO5", "PO4", "PO6", "POz", "O1", "Oz", "O2"] #canais ideais


X_dataset1 = dataset1.pick_channels(ch_names=ch_ideal)._data #pega os dados dos canais ideais
X_dataset2 = dataset2.pick_channels(ch_names=ch_ideal)._data 

X_dataset1 = dataset1.get_data().reshape(dataset1.get_data().shape[0], dataset1.get_data().shape[1] * dataset1.get_data().shape[2]) #reshape para duas dimensoes
X_dataset2 = dataset2.get_data().reshape(dataset2.get_data().shape[0], dataset2.get_data().shape[1] * dataset2.get_data().shape[2])


X_train1, X_test1, y_train1, y_test1 = train_test_split(X_dataset1, y_dataset1, test_size=0.2, random_state=42) #divide os dados em treino e teste
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_dataset2, y_dataset2, test_size=0.2, random_state=42)


n_features_to_select = 9  #numero de features a serem selecionadas


from sklearn.preprocessing import LabelEncoder #transforma os labels em numeros

label_encoder = LabelEncoder()
y_train1 = label_encoder.fit_transform(y_train1) 
y_test1 = label_encoder.transform(y_test1)  
y_train2 = label_encoder.fit_transform(y_train2)
y_test2 = label_encoder.transform(y_test2)


svc1 = SVC(kernel="linear") #cria o classificador
svc2 = SVC(kernel="linear")

rfe1 = RFE(estimator=svc1, step=0.1,n_features_to_select=n_features_to_select) #cria o RFE
rfe2 = RFE(estimator=svc2, step=0.1, n_features_to_select=n_features_to_select)

X_train_selected1 = rfe1.fit_transform(X_train1, y_train1) #seleciona as features
X_test_selected1 = rfe1.transform(X_test1) 
X_train_selected2 = rfe1.fit_transform(X_train2, y_train2) 
X_test_selected2 = rfe1.transform(X_test2)


svc1.fit(X_train_selected1, y_train1) #treina o classificador
svc2.fit(X_train_selected2, y_train2)

y_pred1 = svc1.predict(X_test_selected1) #faz a predicao
y_pred2 = svc2.predict(X_test_selected2)

y_test1 = np.ravel(y_test1) #transforma em array
y_pred1 = np.ravel(y_pred1)

y_test1 = y_test1.reshape(-1, 1) #reshape para duas dimensoes
y_pred1 = y_pred1.reshape(-1, 1)

y_test2 = np.ravel(y_test2) 
y_pred2 = np.ravel(y_pred2)

y_test2 = y_test2.reshape(-1, 1)
y_pred2 = y_pred2.reshape(-1, 1)


accuracia1 = accuracy_score(y_test1, y_pred1) #calcula a acuracia
accuracia2 = accuracy_score(y_test2, y_pred2)

print(f"Acurária para o Dataset 1: {accuracia1 * 100:.2f}%")
print(f"Acurária para o Dataset 2: {accuracia2 * 100:.2f}%")

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\caio-\AppData\Local\Temp\ipykernel_22872\3714709817.py:12: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in an error because the following channel names are missing:
['Pz', 'POz', 'Oz']
Either fix your included names or explicitly pass ordered=False.
  X_dataset1 = dataset1.pick_channels(ch_names=ch_ideal)._data #pega os dados dos canais ideais
C:\Users\caio-\AppData\Local\Temp\ipykernel_22872\3714709817.py:12: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  X_dataset1 = dataset1.pick_channels(ch_names=ch_ideal)._data #pega os dados dos canais ideais
C:\Users\caio-\AppData\Local\Temp\ipykernel_22872\3714709817.py:13: FutureWarning: The default for pick_channels will change from ord

Acurária para o Dataset 1: 0.00%
Acurária para o Dataset 2: 6.25%
